In [ ]:
#Importing required libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
myfile= files.upload()

Saving 1661-0.txt to 1661-0.txt


In [ ]:
#Load and pre-process the data

In [ ]:
file= open("1661-0.txt", "r", encoding= "utf8")

#storing file in a list
lines= []
for i in file:
  lines.append(i)

#converting list to string
data= ""
for i in lines:
  data= " ". join(lines)

#replacing unnecessary stuff with space
data= data.replace("\n","").replace("\r","").replace("\ufeff","").replace('"',"").replace('"',"") #newline, carriage return, unicode character, quote

#remove unnecessary spaces
data= data.split()
data= " ".join(data)
data[:500]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net Title: The Adventures of Sherlock Holmes Author: Arthur Conan Doyle Release Date: November 29, 2002 [EBook #1661] Last Updated: May 20, 2019 Language: English Character set en"

In [ ]:
len(data)

578728

In [ ]:
#applying tokenization

In [ ]:
tokenizer= Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer, open("token.pkl","wb"))

sequence_data= tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[145, 4789, 1, 1020, 4, 128, 34, 45, 611, 2235]

In [ ]:
len(sequence_data) #now each unique word have a numeric representation, filtering punctuations and white spaces

111252

In [ ]:
vocab_size= len(tokenizer.word_index)+1
vocab_size #number of unique words

8931

In [ ]:
sequences= []

for i in range(3, len(sequence_data)):
  words= sequence_data[i-3:i+1]
  sequences.append(words)

print("Length of sequences are: ", len(sequences))
sequences= np.array(sequences)
sequences[:10]

Length of sequences are:  111249


array([[ 145, 4789,    1, 1020],
       [4789,    1, 1020,    4],
       [   1, 1020,    4,  128],
       [1020,    4,  128,   34],
       [   4,  128,   34,   45],
       [ 128,   34,   45,  611],
       [  34,   45,  611, 2235],
       [  45,  611, 2235, 2236],
       [ 611, 2235, 2236,   30],
       [2235, 2236,   30, 1021]])

In [ ]:
X= []
y= []

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])

X= np.array(X)
y= np.array(y)

In [ ]:
print("Data: ",X[:10])
print("Response: ",y[:10])

Data:  [[ 145 4789    1]
 [4789    1 1020]
 [   1 1020    4]
 [1020    4  128]
 [   4  128   34]
 [ 128   34   45]
 [  34   45  611]
 [  45  611 2235]
 [ 611 2235 2236]
 [2235 2236   30]]
Response:  [1020    4  128   34   45  611 2235 2236   30 1021]


In [ ]:
y= to_categorical(y, num_classes= vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
#creating model

In [ ]:
model= Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation= "relu"))
model.add(Dense(vocab_size, activation= "softmax"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 10)             89310     
                                                                 
 lstm_2 (LSTM)               (None, 3, 1000)           4044000   
                                                                 
 lstm_3 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 8931)              8939931   
                                                                 
Total params: 22078241 (84.22 MB)
Trainable params: 22078241 (84.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#plotting the model
#from tensorflow import keras
#from keras.utils import plot_model
#keras.utils.plot_model(model, to_file="plot.png", show_layer_names= True)

In [ ]:
#building model
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint= ModelCheckpoint("next_word.h5", monitor= "loss", verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer= Adam(learning_rate= 0.001))
model.fit(X,y, epochs=70, batch_size=64, callbacks=[checkpoint])


Epoch 1/70
1739/1739 [==============================] - ETA: 0s - loss: 6.4228
Epoch 1: loss improved from inf to 6.42278, saving model to next_word.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1739/1739 [==============================] - 47s 22ms/step - loss: 6.4228
Epoch 2/70
1739/1739 [==============================] - ETA: 0s - loss: 5.7977
Epoch 2: loss improved from 6.42278 to 5.79772, saving model to next_word.h5
1739/1739 [==============================] - 30s 17ms/step - loss: 5.7977
Epoch 3/70
1739/1739 [==============================] - ETA: 0s - loss: 5.4493
Epoch 3: loss improved from 5.79772 to 5.44929, saving model to next_word.h5
1739/1739 [==============================] - 34s 20ms/step - loss: 5.4493
Epoch 4/70
1739/1739 [==============================] - ETA: 0s - loss: 5.1861
Epoch 4: loss improved from 5.44929 to 5.18612, saving model to next_word.h5
1739/1739 [==============================] - 31s 18ms/step - loss: 5.1861
Epoch 5/70
1739/1739 [==============================] - ETA: 0s - loss: 4.9625
Epoch 5: loss improved from 5.18612 to 4.96246, saving model to next_word.h5
1739/1739 [==============================] - 35s 20ms/step - loss: 4.9625
Epoch 

In [ ]:
#predicting
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#loading the model and tokenizer
model= load_model("next_word.h5")
tokenizer= pickle.load(open("token.pkl", "rb"))

def Predict_Next_Words(model, tokenizer, text):
  sequence= tokenizer.texts_to_sequences([text])
  sequence= np.array(sequence)
  preds= np.argmax(model.predict(sequence))
  predicted_word= ""

  for key, value in tokenizer.word_index.items():
    if value==preds:
      predicted_word= key
      break
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text= input("Enter your line: ")

  if text == "0":
    print("Execution completed.....")
    break

  else:
    try:
      text = text.split(" ")
      text = text[-3:]
      print(text)

      Predict_Next_Words(model, tokenizer, text)

    except Exception as e:
      print("Error occurred: ",e)
      continue


Enter your line: extraordinary powers of
['extraordinary', 'powers', 'of']
1/1 [==============================] - 1s 705ms/step
observation
Enter your line: threw across his case of
['his', 'case', 'of']
1/1 [==============================] - 0s 18ms/step
cigars
Enter your line: When I hear you give your
['you', 'give', 'your']
1/1 [==============================] - 0s 20ms/step
casting
Enter your line: since you are interested in these
['interested', 'in', 'these']
1/1 [==============================] - 0s 22ms/step
little


KeyboardInterrupt: ignored